In [1]:
import pandas as pd
import json
from tqdm.notebook import tqdm as tqdm
import swifter
import numpy as np
file_root = 'D:/Career/HKUST(GZ)/5002/dsaa5002_project/'

In [2]:
# Import the A-share list
with open(file_root + 'A_share_list.json', 'r', encoding='utf-8') as file:
    A_share_list = json.load(file)
comp_all = []
for c in A_share_list:
    comp_all.append([c['name'], c['code'], c['fullname']])
companies_df = pd.DataFrame(comp_all, columns=['name', 'code', 'fullname'])

In [7]:
news_all = pd.read_excel(file_root + 'News.xlsx', sheet_name='Sheet1')
cnt_all = len(news_all)

In [4]:
# Clean the news content
news_all['NewsContent'] = news_all['NewsContent'].fillna('').astype(str).str.replace('[\u3000\t\n]', '', regex=True)

## Construct the filter for the news

In [6]:
pd.set_option('display.float_format', None)
news_all.describe()

,NewsID
count,1.037035e+06
mean,5.185180e+05
std,2.993664e+05
min,1.000000e+00
25%,2.592595e+05
50%,5.185180e+05
75%,7.777765e+05
max,1.037035e+06


In [18]:
# Clean the news with no content
cnt_before = len(news_all)
news_filtered = news_all[news_all['NewsContent'] != '']
cnt_after = len(news_filtered)
print('The number of news with no content is reduced from {} to {}, a total reduction of {}'.format(cnt_before, cnt_after, cnt_before - cnt_after))
print('Filter rate = {}'.format((cnt_before - cnt_after) / cnt_all))

The number of news with no content is reduced from 1037035 to 1037035, a total reduction of 0
Filter rate = 0.0


In [19]:
# Clean the news with source from international media
cnt_before = len(news_filtered)
sources_to_filter = ['GCI', 'IFX', 'ACM', 'Admiral Markets', '3TG PLUS', 'ACY稀万国际', 'FBS', 'FxPro', 'FXCM', 'Domino多米乐', 'Activtrades', 'Hotforex', 'HPCforex', 'CMS/VT', 'Ikon Asia', 'IFX', 'www.cnforex.com', 'XFNA', 'NordFX', 'UBEforex', 'XTB', 'NordFX', 'Wind', 'OANDA', 'Markets.com', 'PNKForex', 'RocoForex', 'SFA外汇']
news_filtered = news_filtered[~news_filtered['NewsSource'].isin(sources_to_filter)]
cnt_after = len(news_filtered)
print('The number of news with source from international media is reduced from {} to {}, a total reduction of {}'.format(cnt_before, cnt_after, cnt_before - cnt_after))
print('Filter rate = {}'.format((cnt_before - cnt_after) / cnt_all))

The number of news with source from international media is reduced from 1037035 to 1031250, a total reduction of 5785
Filter rate = 0.005578403814721779


In [20]:
# Clean the news that is not in Chinese
cnt_before = len(news_filtered)
def contains_chinese(text):
    return any(u'\u4e00' <= char <= u'\u9fff' for char in text)
news_filtered = news_filtered[news_filtered['Title'].apply(contains_chinese)]
cnt_after = len(news_filtered)
print('The number of news in English is reduced from {} to {}, a total reduction of {}'.format(cnt_before, cnt_after, cnt_before - cnt_after))
print('Filter rate = {}'.format((cnt_before - cnt_after) / cnt_all))

The number of news in English is reduced from 1031250 to 1022307, a total reduction of 8943
Filter rate = 0.008623624082118734


In [21]:
# Clean the news that is duplicated
cnt_before = len(news_filtered)
news_filtered = news_filtered.drop_duplicates(subset='Title', keep='first')
cnt_after = len(news_filtered)
print('The number of news that is duplicated is reduced from {} to {}, a total reduction of {}'.format(cnt_before, cnt_after, cnt_before - cnt_after))
print('Filter rate = {}'.format((cnt_before - cnt_after) / cnt_all))

The number of news that is duplicated is reduced from 1022307 to 995629, a total reduction of 26678
Filter rate = 0.025725264817484463


## Rough match the news with the A-share list

In [11]:
news_filtered['Title&Content'] = news_filtered.apply(lambda row: row['Title'] + ' ' + row['NewsContent'], axis=1)

In [9]:
name_mapping = {row['fullname']: row['name'] for _, row in companies_df.iterrows()}
name_mapping.update({row['code']: row['name'] for _, row in companies_df.iterrows()})

company_terms = set(companies_df['name']) | set(companies_df['fullname']) | set(companies_df['code'])

def find_companies_optimized(text):
    companies = set()
    for term in company_terms:
        if term in text:
            # 如果 term 是 fullname 或 code，获取对应的 name
            company_name = name_mapping.get(term, term)
            companies.add(company_name)
    return list(companies) if companies else None


In [10]:
news_filtered['Company'] = news_filtered['Title&Content'].swifter.apply(find_companies_optimized)

Pandas Apply:   0%|          | 0/995457 [00:00<?, ?it/s]

In [11]:
# Split the news into two parts
news_to_match = news_filtered[news_filtered['Company'].isna()]
print('The number of news that can be roughly matched is {}'.format(len(news_filtered) - len(news_to_match)))
print('The number of news that cannot be roughly matched is {}'.format(len(news_to_match)))

The number of news that can be roughly matched is 473334
The number of news that cannot be roughly matched is 522123


In [12]:
news_to_match.to_csv(file_root + 'news_to_match.csv', encoding='utf-8-sig', index=False)
news_filtered.to_csv(file_root + 'News_filtered.csv', encoding='utf-8-sig', index=False)

In [4]:
# news_to_match = pd.read_csv(file_root + 'news_to_match.csv', encoding='utf-8-sig')
# news_filtered = pd.read_csv(file_root + 'News_filtered.csv', encoding='utf-8-sig')

In [14]:
# import ast
# news_to_match['Orgs'] = news_to_match['Orgs'].apply(ast.literal_eval)
# news_to_match.head(5)

## NER
Use LAC to do NER

In [16]:
# Initialize LAC, make sure GPU is enabled
from LAC import LAC
lac = LAC(mode='lac', use_cuda=True)
batch_size = 1000
texts = news_to_match['Title&Content']
results = []
for i in tqdm(range(0, len(texts), batch_size)):
    batch = texts[i:i + batch_size]
    processed_batch = lac.run(batch.tolist())
    results.extend(processed_batch)
news_to_match['NER'] = pd.Series(results, index=news_to_match.index)

  0%|          | 0/523 [00:00<?, ?it/s]

C:\Users\Bella\AppData\Local\Temp\ipykernel_22384\968685346.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  news_to_match['NER'] = pd.Series(results, index=news_to_match.index)


In [17]:
news_to_match.to_csv(file_root + 'news_to_match.csv', encoding='utf-8-sig', index=False)

In [18]:
def get_orgs(ner_result):
    orgs = []
    for i, type_w in enumerate(ner_result[1]):
        if type_w == 'ORG':
            orgs.append(ner_result[0][i])
    return orgs

news_to_match['Orgs'] = news_to_match['NER'].apply(get_orgs)

C:\Users\Bella\AppData\Local\Temp\ipykernel_22384\2367639824.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  news_to_match['Orgs'] = news_to_match['NER'].apply(get_orgs)


In [10]:
# Make some cleaning for the companies
# Enlarge the search space
companies_df['name_bk'] = companies_df['name']
special_mappings = {
    '*ST生物':'南华生物',
    'ST生态':'洪湖生态',
    '*ST节能':'神雾节能',
    '*ST环保':'天创信息',
    'ST海洋':'厦门海洋',
    '*ST集成':'成飞集成',
    '*ST数码':'航煤数码'
}

companies_df['name'] = companies_df['name'].map(special_mappings).fillna(companies_df['name'])

In [11]:
terms = ['S*ST', '*ST', 'ST', 'B股']
def remove_special_terms(name, special_terms=terms):
    for term in special_terms:
        name = name.replace(term, '')
    return name.strip()

# 使用DataFrame的apply和lambda函数来检查 name_bk 中是否含有特殊字段
contains_special_terms = companies_df['name_bk'].apply(
    lambda x: any(term in x for term in terms)
)

# 应用这个函数到 'name' 列
companies_df['name'] = companies_df['name'].apply(remove_special_terms)
# 筛选出含有特殊字段的行
companies_with_special_terms = companies_df[contains_special_terms]
companies_with_special_terms

,name,code,fullname,name_bk
897,南华,000660.SZ,广州大通资源开发股份有限公司,*ST南华
904,国重装,601399,国机重型装备集团股份有限公司,ST国重装
967,石化A,000013.SZ,深圳石化工业集团股份有限公司,*ST石化A
970,中侨,000047.SZ,深圳市中侨发展股份有限公司,ST中侨
1012,北讯,002359.SZ,北讯集团股份有限公司,*ST北讯
...,...,...,...,...
4416,山水,600234.SH,山西广和山水文化传播股份有限公司,*ST山水
4419,狮头,600539.SH,太原狮头水泥股份有限公司,ST狮头
4432,坊展,600149.SH,廊坊发展股份有限公司,ST坊展
4484,津滨,000897.SZ,天津津滨发展股份有限公司,*ST津滨


In [52]:
from thefuzz import fuzz
companies_set = set(companies_df['name'])
def match_company(orgs, companies = companies_set, threshold=80):
    matched = set()
    for org in orgs:
        for company in companies:
            similarity = fuzz.ratio(org, company)
            if similarity >= threshold:
                matched.add(company)
    return list(matched) if matched else None

In [ ]:
news_to_match['Company'] = news_to_match['Orgs'].swifter.apply(match_company)

Pandas Apply:   0%|          | 0/522123 [00:00<?, ?it/s]

In [ ]:
news_to_match[['NewsID', 'NewsContent', 'Company']].to_csv(file_root + 'news_to_match_Company.csv', encoding='utf-8-sig', index=False)

In [5]:
# merge the news_to_match and news_filtered
# to match the company found in news_to_match
news_to_match_renamed = news_to_match.rename(columns={'Company': 'MatchedCompany'})
news_filtered = news_filtered.merge(news_to_match_renamed[['NewsID', 'MatchedCompany']], on='NewsID', how='left')
news_filtered['Company'] = news_filtered.apply(
    lambda row: row['MatchedCompany'] if pd.isna(row['Company']) else row['Company'],
    axis=1
)
news_filtered.drop(columns=['MatchedCompany'], inplace=True)
news_filtered.head(5)

,NewsID,Title,NewsContent,NewsSource,Title&Content,Company
0,1,建设银行原董事长张恩照一审被判15年,本报记者 田雨 李京华 中国建设银行股份有限公司原董事长张恩照受贿案３日一审宣判，北京...,中国证券报,建设银行原董事长张恩照一审被判15年 本报记者 田雨 李京华 中国建设银行股份有限公司...,['建设银行']
1,2,农行信用卡中心搬到上海滩,中国农业银行信用卡中心由北京搬到上海了！ 农行行长杨明生日前在信用卡中心揭牌仪式上表示，此...,人民日报,农行信用卡中心搬到上海滩 中国农业银行信用卡中心由北京搬到上海了！ 农行行长杨明生日前在信...,['农业银行']
2,3,外运发展：价值型蓝筹股补涨要求强烈,在新基金快速发行以及申购资金回流的情况下，市场总体上呈现资金流动性过剩格局，考虑到现阶段权重...,杭州新希望,外运发展：价值型蓝筹股补涨要求强烈 在新基金快速发行以及申购资金回流的情况下，市场总体上呈现...,"['中国国航', '外运发展']"
3,4,胜利股份：稳步走强形成标准上升通道,胜利股份（000407）公司子公司填海造地2800亩，以青岛的地价估算，静态价值在10亿元左...,源达投资,胜利股份：稳步走强形成标准上升通道 胜利股份（000407）公司子公司填海造地2800亩，以...,['胜利股份']
4,5,[港股快讯]恒指收市报18960点 成交467亿港元,全景网11月30日讯 外围股市造好，带动港股今早造好，恒指高开后反覆上升，最高升252点，曾...,全景网,[港股快讯]恒指收市报18960点 成交467亿港元 全景网11月30日讯 外围股市造好，带...,NaN


In [9]:
news_filtered

,NewsID,Title,NewsContent,NewsSource,Title&Content,Company
0,1,建设银行原董事长张恩照一审被判15年,本报记者 田雨 李京华 中国建设银行股份有限公司原董事长张恩照受贿案３日一审宣判，北京...,中国证券报,建设银行原董事长张恩照一审被判15年 本报记者 田雨 李京华 中国建设银行股份有限公司...,['建设银行']
1,2,农行信用卡中心搬到上海滩,中国农业银行信用卡中心由北京搬到上海了！ 农行行长杨明生日前在信用卡中心揭牌仪式上表示，此...,人民日报,农行信用卡中心搬到上海滩 中国农业银行信用卡中心由北京搬到上海了！ 农行行长杨明生日前在信...,['农业银行']
2,3,外运发展：价值型蓝筹股补涨要求强烈,在新基金快速发行以及申购资金回流的情况下，市场总体上呈现资金流动性过剩格局，考虑到现阶段权重...,杭州新希望,外运发展：价值型蓝筹股补涨要求强烈 在新基金快速发行以及申购资金回流的情况下，市场总体上呈现...,"['中国国航', '外运发展']"
3,4,胜利股份：稳步走强形成标准上升通道,胜利股份（000407）公司子公司填海造地2800亩，以青岛的地价估算，静态价值在10亿元左...,源达投资,胜利股份：稳步走强形成标准上升通道 胜利股份（000407）公司子公司填海造地2800亩，以...,['胜利股份']
7,8,"南风化工：钾肥三雄之一 被""中化""相中",由于全球最大的俄罗斯Uralkaly钾矿被淹，产量大减，同时满洲里口岸铁路在修复线，导致中俄...,银河证券,"南风化工：钾肥三雄之一 被""中化""相中 由于全球最大的俄罗斯Uralkaly钾矿被淹，产量大...",['冠农股份']
...,...,...,...,...,...,...
995452,1037031,亿华通：公司电解槽相关产品目前还处于产品的研发及测试阶段 尚未实现批量销售,每经AI快讯，有投资者在投资者互动平台提问：请问公司目前有没有电解槽产能，规划情况能否详细介...,每日经济新闻,亿华通：公司电解槽相关产品目前还处于产品的研发及测试阶段 尚未实现批量销售 每经AI快讯，有...,['亿华通']
995453,1037032,依米康：接受中泰证券调研,依米康（SZ 300249，收盘价：10.38元）发布公告称，2023年10月12日，依米康...,每日经济新闻,依米康：接受中泰证券调研 依米康（SZ 300249，收盘价：10.38元）发布公告称，20...,"['中泰证券', '依米康']"
995454,1037033,天风证券给予中核科技买入评级 核电行业景气上行 公司有望乘风而起,天风证券10月13日发布研报称，给予中核科技（000777.SZ，最新价：13.03元）买入...,每日经济新闻,天风证券给予中核科技买入评级 核电行业景气上行 公司有望乘风而起 天风证券10月13日发布研...,"['天风证券', '中核科技']"
995455,1037034,海特生物：公司在抗癌药CPT获批后 会考虑适时开展CPT在海外的临床并谋求上市,有投资者提问：抗癌药CPT获批后，公司是否应该按照股权协议继续收购沙东股权，适应症为MM的C...,界面新闻,海特生物：公司在抗癌药CPT获批后 会考虑适时开展CPT在海外的临床并谋求上市 有投资者提问...,['海特生物']


In [8]:
cnt_before = len(news_filtered)
news_filtered.dropna(subset=['Company'], inplace=True)
cnt_after = len(news_filtered)
print('The number of news that cannot be matched is reduced from {} to {}, a total reduction of {}'.format(cnt_before, cnt_after, cnt_all - cnt_after))

The number of news that cannot be matched is reduced from 995457 to 496053, a total reduction of 540982


In [12]:
back_mappings = {}
for _, row in tqdm(companies_df.iterrows()):
    back_mappings[row['name']] = row['name_bk']
news_filtered['Company'] = news_filtered['Company'].map(back_mappings).fillna(news_filtered['Company'])

0it [00:00, ?it/s]

In [ ]:
def list_to_string(lst):
    # turn the list of companies into a string
    return ','.join(lst)
news_filtered = news_filtered['Company'].apply(list_to_string)

In [13]:
news_filtered.to_csv(file_root + 'News_matched.csv', encoding='utf-8-sig', index=True)